In [36]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [37]:
df = pd.read_csv("org_info.csv")
df

,Unnamed: 0,Organisation name:,Founding year:,Country:,Total number of staff:,Resources dedicated to microfinance:,Percentage of microfinance activities:,Type of organisation:,,Contact person:,Address:,Email:,Website:,Phone:,Fax:,Mission:,Type of intervention:,Countries of intervention:,Areas of intervention:,Main competences:,Please indicate any innovations in your microfinance program planned for the coming year:,Foundation year:,Country of incorporation:,Environmental Microfinance Project Description:
0,1,Academy of German Cooperatives / Akademie Deutscher Genossenschaften,1973,Germany,300,8,Between 0 and 20%,Non profit,"Consultants and support services (such as rating, IT or law firms)",Ms. Isabelle Katthagen,Schloss Montabaur D-56410MontabaurGermany,Isabelle_Katthagen@adgonline.de,www.adg-international.com,+49 2602 14 158,+49 (2) 602 14 95 1 58,Managerial education and financial training provider for cooperative and other financial institutions.\n,"Subsidies in staff time (e.g. technical assistance/capacity building, evaluation)Technical assistance/capacity building, Evaluations: consultants in technical assistance projects on behalf of international donor organizations",AfricaAsiaEuropeSouth AmericaBrazilOceania,"Macro level: Prudential regulation, Supervision, Supporting networks and associations, Strengthening national/regional microfinance support sector, Capacity building (focused on all MFIs)Intermediate level: Rural finance, Value chain development, Business development services, Educational FinanceMicro level: Financial product development and innovation, Energy products, or other products related to environmental issues, MFI operations (e.g. general management, governance, finance and accounting, staff training, recuiting), Risk management, Management information systems, Educational Finance Product DevelopmentOther themes: Sector mapping, Feasibility studies","Academy of German Cooperatives has special knowledge/expertise on development of management competencies (strategy, marketing, product development, HRD, leadership & coaching..), bank management (risk, etc.); supervision and internal/external auditing of managers and specialists working in MFIs. \n",NaN,NaN,NaN,NaN
1,2,Advans Group,2005,France,61,61,100%,For profit,"Consultants and support services (such as rating, IT or law firms), Financial institution",Katherine Brown,39 Rue La FayetteParisFrance,kbrown@advansgroup.com,www.advansgroup.com,01 53 32 75 75,NaN,"Advans mission is to respond to the need for financial services of Micro, Small and Medium-sized Enterprises (MSMEs) and other populations who have ill-adapted, limited or no access to formal financial services through providing tailored financial services in a sustainable and responsible manner.\nOur vision is to build a model group of financial institutions which contribute to strengthening local businesses, creating and sustaining jobs and improving clients' living standards in order to foster private sector-led economic and social development in Africa, the Middle-East and Asia.\n",NaN,"AfricaCameroonCongo, The Democratic Republic of theCôte d'IvoireGhanaNigeriaTanzania, United Republic ofTunisiaAsiaCambodiaMyanmarPakistan",NaN,NaN,"Over the next five years, the Advans group will be focusing on:\n\nUsing new technologies to improve quality of service and productivity, such as: remote data capture solutions for commercial staff, testing possibility of scoring for micro-loans\nDiversifying the credit offering by further developing agri-lending, SME lending and personal lending (housing finance, education finance, emergency nano-loans)\nFurther developing deposits & financial services to establish a strong retail deposit basis\nDeploying a multi-channel strategy for proximity and convenience, using ATM cards, mini cash counters, cashless points of services, field tellers, mobile banking, mobile wallets, agency banking... \nDeveloping the brand and focusing on developing its client centred culture a

In [38]:
# Drop non informative column
df = df.drop(columns=["Unnamed: 0"])

In [39]:
# Renaming columns because of bad formatting
df.columns = ['Organisation name', 'Founding year', 'Country',
       'Total number of staff', 'Resources dedicated to microfinance',
       'Percentage of microfinance activities', 'Profit',
       'Type of organisation', 'Contact person', 'Address', 'Email', 'Website', 'Phone',
       'Fax', 'Mission', 'Type of intervention',
       'Countries of intervention', 'Areas of intervention',
       'Main competences',
       'Please indicate any innovations in your microfinance program planned for the coming year',
       'Foundation year', 'Country of incorporation',
       'Environmental Microfinance Project Description']

In [40]:
# Organisation name is not useful for analytics. It's better used as row ID.
df = df.set_index('Organisation name')

In [41]:
# Remove info not useful for analytics
df = df.drop(columns=["Contact person", "Address", "Email", "Website", "Phone", "Fax"])
df.head()

,Founding year,Country,Total number of staff,Resources dedicated to microfinance,Percentage of microfinance activities,Profit,Type of organisation,Mission,Type of intervention,Countries of intervention,Areas of intervention,Main competences,Please indicate any innovations in your microfinance program planned for the coming year,Foundation year,Country of incorporation,Environmental Microfinance Project Description
Organisation name,,,,,,,,,,,,,,,,
Academy of German Cooperatives / Akademie Deutscher Genossenschaften,1973,Germany,300,8,Between 0 and 20%,Non profit,"Consultants and support services (such as rating, IT or law firms)",Managerial education and financial training provider for cooperative and other financial institutions.\n,"Subsidies in staff time (e.g. technical assistance/capacity building, evaluation)Technical assistance/capacity building, Evaluations: consultants in technical assistance projects on behalf of international donor organizations",AfricaAsiaEuropeSouth AmericaBrazilOceania,"Macro level: Prudential regulation, Supervision, Supporting networks and associations, Strengthening national/regional microfinance support sector, Capacity building (focused on all MFIs)Intermediate level: Rural finance, Value chain development, Business development services, Educational FinanceMicro level: Financial product development and innovation, Energy products, or other products related to environmental issues, MFI operations (e.g. general management, governance, finance and accounting, staff training, recuiting), Risk management, Management information systems, Educational Finance Product DevelopmentOther themes: Sector mapping, Feasibility studies","Academy of German Cooperatives has special knowledge/expertise on development of management competencies (strategy, marketing, product development, HRD, leadership & coaching..), bank management (risk, etc.); supervision and internal/external auditing of managers and specialists working in MFIs. \n",NaN,NaN,NaN,NaN
Advans Group,2005,France,61,61,100%,For profit,"Consultants and support services (such as rating, IT or law firms), Financial institution","Advans mission is to respond to the need for financial services of Micro, Small and Medium-sized Enterprises (MSMEs) and other populations who have ill-adapted, limited or no access to formal financial services through providing tailored financial services in a sustainable and responsible manner.\nOur vision is to build a model group of financial institutions which contribute to strengthening local businesses, creating and sustaining jobs and improving clients' living standards in order to foster private sector-led economic and social development in Africa, the Middle-East and Asia.\n",NaN,"AfricaCameroonCongo, The Democratic Republic of theCôte d'IvoireGhanaNigeriaTanzania, United Republic ofTunisiaAsiaCambodiaMyanmarPakistan",NaN,NaN,"Over the next five years, the Advans group will be focusing on:\n\nUsing new technologies to improve quality of service and productivity, such as: remote data capture solutions for commercial staff, testing possibility of scoring for micro-loans\nDiversifying the credit offering by further developing agri-lending, SME lending and personal lending (housing finance, education finance, emergency nano-loans)\nFurther developing deposits & financial services to establish a strong retail deposit basis\nDeploying a multi-channel strategy for proximity and convenience, using ATM cards, mini cash counters, cashless points of services, field tellers, mobile banking, mobile wallets, agency banking... \nDeveloping the brand and focusing on developing its client centred culture and organisation\n\n \n",NaN,NaN,NaN
AFC Agriculture & Finance Consultants GmbH,1973,Germany,30,3,Between 0 and 20%,For profit,"Consultants and support services (such as rating, IT or law firms)","Our mission: AFC provides competent consulting services to support agricultural, agribusiness and financial sector development in cooperati

In [42]:
# Show null values ratio for every column
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True, ascending= False)
missing_value_df

,percent_missing
Foundation year,99.009901
Country of incorporation,99.009901
Environmental Microfinance Project Description,99.009901
Please indicate any innovations in your microfinance program planned for the coming year,44.554455
Main competences,32.673267
Type of intervention,22.772277
Areas of intervention,17.821782
Resources dedicated to microfinance,14.851485
Total number of staff,3.960396
Founding year,0.000000


In [43]:
# Remove columns mostly filled with NaN
df = df.drop(columns=['Foundation year', 'Country of incorporation',
       'Environmental Microfinance Project Description'])

In [44]:
# Inspect column dtypes
df.dtypes

Founding year                                                                               object
Country                                                                                     object
Total number of staff                                                                       object
Resources dedicated to microfinance                                                         object
Percentage of microfinance activities                                                       object
Profit                                                                                      object
Type of organisation                                                                        object
Mission                                                                                     object
Type of intervention                                                                        object
Countries of intervention                                                                   object
Areas of i

In [45]:
# Some elements have additional text/specification after the Founding Year value
# We get rid of it
df['Founding year'] = df['Founding year'].str[0:4]

In [46]:
# Change numerical values to Nullable Integers (must convert to 'float' before 
# doing so, becuase of a bug)

df['Founding year'] = df['Founding year'].apply(pd.to_numeric, errors='coerce').astype(pd.Int64Dtype())
df['Total number of staff'] = df['Total number of staff'].apply(pd.to_numeric, errors='coerce').astype(pd.Int64Dtype())
df['Resources dedicated to microfinance'] = df['Resources dedicated to microfinance'].apply(pd.to_numeric, errors='coerce').astype(pd.Int64Dtype())

In [47]:
# Set categorical and string classes
df = df.astype({'Country': "category",
                'Profit': "category"})
#           'Type of organisation': "category", "Mission": str})
df.dtypes

Founding year                                                                                  Int64
Country                                                                                     category
Total number of staff                                                                          Int64
Resources dedicated to microfinance                                                            Int64
Percentage of microfinance activities                                                         object
Profit                                                                                      category
Type of organisation                                                                          object
Mission                                                                                       object
Type of intervention                                                                          object
Countries of intervention                                                                  

In [48]:
# Change "percentage of microfinance activities" into a numerical (float) value:
# the range midpoint.
df['Percentage of microfinance activities'] = df['Percentage of microfinance activities'].replace({'Between ': '', '%': ''}, regex=True)
df[['From','To']] = df['Percentage of microfinance activities'].str.split(' and ', expand=True)
df['To'].fillna(df.From, inplace=True)
df['Percentage of microfinance activities'] = (df.From.apply(lambda x: float(x)) + df.To.apply(lambda x: float(x)))/ 2
df = df.drop(columns=["From", "To"])
df.head()

,Founding year,Country,Total number of staff,Resources dedicated to microfinance,Percentage of microfinance activities,Profit,Type of organisation,Mission,Type of intervention,Countries of intervention,Areas of intervention,Main competences,Please indicate any innovations in your microfinance program planned for the coming year
Organisation name,,,,,,,,,,,,,
Academy of German Cooperatives / Akademie Deutscher Genossenschaften,1973,Germany,300,8,10.0,Non profit,"Consultants and support services (such as rating, IT or law firms)",Managerial education and financial training provider for cooperative and other financial institutions.\n,"Subsidies in staff time (e.g. technical assistance/capacity building, evaluation)Technical assistance/capacity building, Evaluations: consultants in technical assistance projects on behalf of international donor organizations",AfricaAsiaEuropeSouth AmericaBrazilOceania,"Macro level: Prudential regulation, Supervision, Supporting networks and associations, Strengthening national/regional microfinance support sector, Capacity building (focused on all MFIs)Intermediate level: Rural finance, Value chain development, Business development services, Educational FinanceMicro level: Financial product development and innovation, Energy products, or other products related to environmental issues, MFI operations (e.g. general management, governance, finance and accounting, staff training, recuiting), Risk management, Management information systems, Educational Finance Product DevelopmentOther themes: Sector mapping, Feasibility studies","Academy of German Cooperatives has special knowledge/expertise on development of management competencies (strategy, marketing, product development, HRD, leadership & coaching..), bank management (risk, etc.); supervision and internal/external auditing of managers and specialists working in MFIs. \n",NaN
Advans Group,2005,France,61,61,100.0,For profit,"Consultants and support services (such as rating, IT or law firms), Financial institution","Advans mission is to respond to the need for financial services of Micro, Small and Medium-sized Enterprises (MSMEs) and other populations who have ill-adapted, limited or no access to formal financial services through providing tailored financial services in a sustainable and responsible manner.\nOur vision is to build a model group of financial institutions which contribute to strengthening local businesses, creating and sustaining jobs and improving clients' living standards in order to foster private sector-led economic and social development in Africa, the Middle-East and Asia.\n",NaN,"AfricaCameroonCongo, The Democratic Republic of theCôte d'IvoireGhanaNigeriaTanzania, United Republic ofTunisiaAsiaCambodiaMyanmarPakistan",NaN,NaN,"Over the next five years, the Advans group will be focusing on:\n\nUsing new technologies to improve quality of service and productivity, such as: remote data capture solutions for commercial staff, testing possibility of scoring for micro-loans\nDiversifying the credit offering by further developing agri-lending, SME lending and personal lending (housing finance, education finance, emergency nano-loans)\nFurther developing deposits & financial services to establish a strong retail deposit basis\nDeploying a multi-channel strategy for proximity and convenience, using ATM cards, mini cash counters, cashless points of services, field tellers, mobile banking, mobile wallets, agency banking... \nDeveloping the brand and focusing on developing its client centred culture and organisation\n\n \n"
AFC Agriculture & Finance Consultants GmbH,1973,Germany,30,3,10.0,For profit,"Consultants and support services (such as rating, IT or law firms)","Our mission: AFC provides competent consulting services to support agricultural, agribusiness and financial sector development in cooperation with public and private sector organizations in developing and transition countries.\nOur vision: AFC continue to be recognised by all st

In [49]:
# Deal with inclusive, multiclass variables

cols = ['Africa', 'Asia','Europe', 'Oceania', 'South America', 'North America']
ALL = '|Programmes open to all regions.'
for col in cols:
    df[col] = np.where(df['Countries of intervention'].str.contains(col+ALL, regex=True), 1, 0)
df = df.drop(columns=['Countries of intervention'])

## TODO: Type of organisation, Type of intervention, Areas of intervention
## Must find a way to extract classes from textual description

## Proof of concept for Areas of intervention: Create "Macro Level", "Intermediate Level", 
## "Micro Level", "Other Themes" columns and for every row extract details of that level from
## "Areas of intervention" and put them in the related column. Then we will decide what to do
## with those details

## Proof of concept for Type of organisation: create a column for every type of organization found in
## "Type of organisation", and assign each organization to its types. Be wary that a single organisation
## can be of many types.

## Type of intervention: same as Type of organisation. Beware that ToIs appear like this in the dataframe:
## "LoansSubsidies in staff time (e.g. technical assistance/capacity building, evaluation)" <- BAD FORMATTING
## It means this org. has two types of interventions: "Loans" and "Subsidies in staff time". 



df.head()

,Founding year,Country,Total number of staff,Resources dedicated to microfinance,Percentage of microfinance activities,Profit,Type of organisation,Mission,Type of intervention,Areas of intervention,Main competences,Please indicate any innovations in your microfinance program planned for the coming year,Africa,Asia,Europe,Oceania,South America,North America
Organisation name,,,,,,,,,,,,,,,,,,
Academy of German Cooperatives / Akademie Deutscher Genossenschaften,1973,Germany,300,8,10.0,Non profit,"Consultants and support services (such as rating, IT or law firms)",Managerial education and financial training provider for cooperative and other financial institutions.\n,"Subsidies in staff time (e.g. technical assistance/capacity building, evaluation)Technical assistance/capacity building, Evaluations: consultants in technical assistance projects on behalf of international donor organizations","Macro level: Prudential regulation, Supervision, Supporting networks and associations, Strengthening national/regional microfinance support sector, Capacity building (focused on all MFIs)Intermediate level: Rural finance, Value chain development, Business development services, Educational FinanceMicro level: Financial product development and innovation, Energy products, or other products related to environmental issues, MFI operations (e.g. general management, governance, finance and accounting, staff training, recuiting), Risk management, Management information systems, Educational Finance Product DevelopmentOther themes: Sector mapping, Feasibility studies","Academy of German Cooperatives has special knowledge/expertise on development of management competencies (strategy, marketing, product development, HRD, leadership & coaching..), bank management (risk, etc.); supervision and internal/external auditing of managers and specialists working in MFIs. \n",NaN,1,1,1,1,1,0
Advans Group,2005,France,61,61,100.0,For profit,"Consultants and support services (such as rating, IT or law firms), Financial institution","Advans mission is to respond to the need for financial services of Micro, Small and Medium-sized Enterprises (MSMEs) and other populations who have ill-adapted, limited or no access to formal financial services through providing tailored financial services in a sustainable and responsible manner.\nOur vision is to build a model group of financial institutions which contribute to strengthening local businesses, creating and sustaining jobs and improving clients' living standards in order to foster private sector-led economic and social development in Africa, the Middle-East and Asia.\n",NaN,NaN,NaN,"Over the next five years, the Advans group will be focusing on:\n\nUsing new technologies to improve quality of service and productivity, such as: remote data capture solutions for commercial staff, testing possibility of scoring for micro-loans\nDiversifying the credit offering by further developing agri-lending, SME lending and personal lending (housing finance, education finance, emergency nano-loans)\nFurther developing deposits & financial services to establish a strong retail deposit basis\nDeploying a multi-channel strategy for proximity and convenience, using ATM cards, mini cash counters, cashless points of services, field tellers, mobile banking, mobile wallets, agency banking... \nDeveloping the brand and focusing on developing its client centred culture and organisation\n\n \n",1,1,0,0,0,0
AFC Agriculture & Finance Consultants GmbH,1973,Germany,30,3,10.0,For profit,"Consultants and support services (such as rating, IT or law firms)","Our mission: AFC provides competent consulting services to support agricultural, agribusiness and financial sector development in cooperation with public and private sector organizations in developing and transition countries.\nOur vision: AFC continue to be recognised by all stakeholders as a reliable company delivering sustainable solutions to projects in agriculture, agribusiness and financial secto